In [22]:
import pandas as pd
import os
import akvut

query = """
query MyQuery($id: ID!) {
  course(id: $id) {
    enrollmentsConnection {
      nodes {
        user {
          sisId
        }
        type
      }
    }
    sisId
  }
}
"""

# Lage liste for å deaktivere alle i eit emne
Hent alle i emnet i dag, og lag ei CSV-fil som vi kan importere med SIS-IMPORT (etter at eg haar rydda i den).

In [23]:
variable = {'id': '26760'}
resultat = akvut.query_canvas_graphql(query, variable)

In [34]:
data = resultat['data']['course']['enrollmentsConnection']['nodes']
liste1 = []
liste2 = []
liste3 = []
for d in data:
    liste1.append([d['user']['sisId'], d['type']])
    liste2.append([d['user']['sisId'], d['type'], resultat['data']['course']['sisId'], 'completed'])
    liste3.append([d['user']['sisId'], d['type'], resultat['data']['course']['sisId'], 'active'])
personar = pd.DataFrame(liste1, columns=['sisId', 'type'])
deaktiver = pd.DataFrame(liste2, columns=['sisId', 'type', 'course', 'workflow_state'])
deaktiver.to_csv("deaktiver.csv", index=False)
aktiver = pd.DataFrame(liste3, columns=['sisId', 'type', 'course', 'workflow_state'])
aktiver.to_csv("aktiver.csv", index=False)